# Browsing Books - A Recomendation Engine for Books
For CMPT3520 Machine Learning II <br/>
Annabell Rodriguez, Laura Brin, Sandra Alex

## Introduction

### Business Problem

### Evaluation Metrics

## Browsing Books

### Loading data

In [ ]:
#Loading Libraries
from __future__ import print_function

import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)

# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

# Install Altair and activate its colab renderer.
#print("Installing Altair...")
#!pip install git+git://github.com/altair-viz/altair.git
#!pip install altair vega_datasets
import altair as alt
alt.data_transformers.enable('default', max_rows=None)
alt.renderers.enable('colab')
#print("Done installing Altair.")

In [ ]:
#Loading Dataset
from ast import literal_eval
 
books_small = pd.read_csv('https://raw.githubusercontent.com/malcolmosh/goodbooks-10k/master/books_enriched.csv', index_col=[0],dtype={"isbn":object}, converters={"genres": literal_eval})
#note books_enriched.csv is a modified version of books.csv from goodbooks-10k dataset
users = pd.read_csv("Datasets\\Users.csv")
ratings= pd.read_csv("Datasets\\Ratings.csv")
books_large = pd.read_csv("Datasets\\Books.csv",dtype={"ISBN":object},low_memory=False)
#books3 = pd.read_csv("Datasets\\books_enriched.csv",dtype={"isbn":object})
#gr_books=pd.read_csv("Datasets\\Goodreads_books_with_genres.csv",dtype={"isbn":object})


### Books

In [ ]:
books_small.head()

In [ ]:
books_small.columns

In [ ]:
books_small.drop(["book_id","best_book_id","description","goodreads_book_id","ratings_1","ratings_2","ratings_3","ratings_4","ratings_5","ratings_count","work_id","work_ratings_count","work_text_reviews_count","authors_2"],axis=1,inplace=True, errors='ignore')

In [ ]:
books_small.dtypes

In [ ]:
# number of books by number of genres
books_small['genres'].value_counts().groupby(len).sum()

In [ ]:
books_small['genres']

In [ ]:
books_small["isbn"].isin(ratings["ISBN"]).value_counts()

In [ ]:
books_small["isbn"].isin(books_large["ISBN"]).value_counts()

In [ ]:
books_large.head()

In [ ]:
books_small["original_title"].isin(books_large["Book-Title"]).value_counts()

In [ ]:
books_large["Book-Title"].isin(books_small["original_title"]).value_counts()

Combining datasets

In [ ]:
all_books=pd.merge(books_small,books_large,how="inner",left_on="original_title",right_on="Book-Title")
all_books=all_books[all_books['language_code'] == 'eng']
all_books.head()

In [ ]:
all_books.drop(['original_title','index','title','isbn13','books_count','publishDate','isbn','language_code','Book-Author','Year-Of-Publication','Publisher','image_url','small_image_url','Image-URL-S','Image-URL-M','Image-URL-L'],axis=1,inplace=True, errors='ignore')

In [ ]:
all_books["ISBN"].isin(ratings["ISBN"]).value_counts()

In [ ]:
# Run Only Once!
all_books["average_rating"]=all_books["average_rating"].apply(lambda x: x*2)

In [ ]:
all_books.head()

In [ ]:
#adding column for consecutive number by duplicate book-title and year
all_books['duplicate_count'] = all_books.groupby(['Book-Title','original_publication_year', 'authors']).cumcount()+1
all_books.head()

In [ ]:
all_books.shape

### Users

In [ ]:
users.head()

In [ ]:
users['city'],users['state'],users['country']=users["Location"].str.split(",",2).str
users.drop(['Location'],axis=1,inplace=True, errors='ignore')
users.head()

In [ ]:
users["country"].fillna("Unknown", inplace = True)
users["state"].fillna("Unknown", inplace = True)
users["city"].fillna("Unknown", inplace = True)

In [ ]:
list=users["country"].value_counts().where(users["country"].value_counts()>7500,other="Other")
print(list)


In [ ]:
users["new_country"]=users["country"].apply(lambda x: x if list[x]!="Other" else "Other")
users["new_country"].value_counts()


In [ ]:
state_list=users["state"].value_counts().where(users["state"].value_counts()>5000,other="Other")
users["new_state"]=users["state"].apply(lambda x: x if state_list[x]!="Other" else "Other")
users["new_state"].value_counts()

In [ ]:
city_list=users["city"].value_counts().where(users["city"].value_counts()>1500,other="Other")
users["new_city"]=users["city"].apply(lambda x: x if city_list[x]!="Other" else "Other")
users["new_city"].value_counts()

In [ ]:
users.describe()

In [ ]:
import random
random.seed(42)

f=lambda x: x if x<100 else round(random.randint(24,44))
users['Age']=users['Age'].apply(f)
users.describe()

In [ ]:
users['Age'].value_counts()

In [ ]:
users.describe(include=[object])

In [ ]:
users.drop(['city','state','country'],axis=1,inplace=True, errors='ignore')

### Ratings

In [ ]:
ratings.head()

In [ ]:
ratings.shape

In [ ]:
ratings_books=pd.merge(ratings,all_books,how="inner",left_on="ISBN",right_on="ISBN")
ratings_books=pd.merge(ratings_books,all_books[all_books['duplicate_count'] == 1],how="inner",left_on=["Book-Title","original_publication_year","authors"],right_on=["Book-Title","original_publication_year","authors"])
ratings_books

In [ ]:
ratings_books['ISBN']=ratings_books.apply(lambda x: x['ISBN_y'] if x['duplicate_count_x']>1 and x['duplicate_count_y']==1 else x['ISBN_x'],axis=1)
ratings_books

In [ ]:
#removing when duplicate_count greater than 1
all_books=all_books[all_books['duplicate_count'] == 1]
#removing column duplicate_count
all_books.drop(['duplicate_count'],axis=1,inplace=True, errors='ignore')
all_books.reset_index(drop=True, inplace=True)
all_books.shape

In [ ]:
all_books['Book_id']=all_books.index + 1

In [ ]:
ratings_books.shape

In [ ]:
#substituting ISBN in ratings_books with book_id
ratings_books=pd.merge(ratings_books,all_books,how="left",left_on="ISBN",right_on="ISBN")


In [ ]:
# dropping columns except User-ID	ISBN	Book-Rating
columns_to_drop = [c for c in ratings_books.columns if c not in ['User-ID', 'Book_id','Book-Rating']]
ratings_books.drop(columns_to_drop ,axis=1,inplace=True, errors='ignore')
ratings_books.duplicated().value_counts()

In [ ]:
ratings_books.drop_duplicates(inplace=True)
ratings_books.shape

In [ ]:
ratings_books.head(-5)

In [ ]:
all_books.head()

In [ ]:
ratings = ratings_books.copy()

### Encoding

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
# create the MultiLabelBinarizer object
mlb = MultiLabelBinarizer()

# one-hot encode
all_books_encoded = pd.DataFrame(mlb.fit_transform(all_books['genres']), columns=mlb.classes_, index=all_books.index)

all_books_final = pd.concat([all_books, all_books_encoded], axis=1)
all_books_final.head()

In [ ]:
genre_occurences=all_books_encoded.sum().to_dict()
genres=genre_occurences.keys()

### Visuals

To visualize genres, books are assigned a random genre from their genre list

In [ ]:
filter=lambda x: random.choice(x)
all_books_final['genre_rnd']=all_books_final['genres'].apply(filter)

In [ ]:
all_books_final['genre_rnd'].value_counts()

In [ ]:
book_ratings = all_books_final.merge(
    ratings
    .groupby("Book_id", as_index=False)
    .agg({'Book-Rating': ['count', 'mean']})
    .flatten_cols(),
    on='Book_id')

genre_filter = alt.selection_multi(fields=['genre_rnd'])

genre_chart = alt.Chart().mark_bar().encode(
    x="count()",
    y=alt.Y('genre_rnd'),
    color=alt.condition(
        genre_filter,
        alt.Color("genre_rnd:N"),
        alt.value('lightgray'))
).properties(height=600, selection=genre_filter)

In [ ]:
(book_ratings[['Book_id', 'Book-Rating count', 'Book-Rating mean']]
 .sort_values('Book-Rating count', ascending=False)
 .head(10))

In [ ]:
def filtered_hist(field, label, filter):
  """Creates a layered chart of histograms.
  The first layer (light gray) contains the histogram of the full data, and the
  second contains the histogram of the filtered data.
  Args:
    field: the field for which to generate the histogram.
    label: String label of the histogram.
    filter: an alt.Selection object to be used to filter the data.
  """
  base = alt.Chart().mark_bar().encode(
      x=alt.X(field, bin=alt.Bin(maxbins=10), title=label),
      y="count()",
  ).properties(
      width=300,
  )
  return alt.layer(
      base.transform_filter(filter),
      base.encode(color=alt.value('lightgray'), opacity=alt.value(.7)),
  ).resolve_scale(y='independent')

In [ ]:
# Display the number of ratings and average rating per book.
alt.hconcat(
    filtered_hist('Book-Rating count', '# ratings / book', genre_filter),
    filtered_hist('Book-Rating mean', 'mean rating', genre_filter),
    genre_chart,
    data=book_ratings)

## Content-Based Filtering

## Collaborative Filtering-Matrix Factorization

In [ ]:
# Create a dictionary of user and movie ids
user_ids = dict(zip(ratings['User-ID'].unique(), range(len(ratings['User-ID'].unique()))))
book_ids = dict(zip(ratings['ISBN'].unique(), range(len(ratings['ISBN'].unique()))))

### Sparse Representation of the Ratings Matrix

The next 5 code blocks contain functions taken from Google's Machine Learning Recomendation Systems Colab document. <br/>
* The first function can be called to split the dataframe into 2 subsets, one for train and one for test. This uses the sample function rather than a train_test_split, and the fraction can be selected when called.
* The second functon is for creating the sparse tensor from the ratings dataframe. As the rated book list contains almost 8000 entries, most books will be unrated by most users. To reduce matrix size, a sparse tensor can be utilized  <br/>
* The third function calculates the mean squared error when given the sparse matrix A and two embedding matrices (U,V). Predictions are made by matrix multiplying the embedding tensors and mapping them to the sparse matrix space. The gather_nd function for TensorFlow combines slices from the new matrix and saves them to match the shape specified by the indices (in this example the size of the sparse matrix). A loss calculation is then performed to evaluate the prediction against the true values recorded in the sparse matrix.<br/>
* The fourth code block defines a helper class for creating the Collaborative Filtering Model. <br/>
* The fifth block builds the Collaborative Filtering Model

In [ ]:
# Utility to split the data into training and test sets.
def split_dataframe(df, holdout_fraction=0.1):
  """Splits a DataFrame into training and test sets.
  Args:
    df: a dataframe.
    holdout_fraction: fraction of dataframe rows to use in the test set.
  Returns:
    train: dataframe for training
    test: dataframe for testing
  """
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]
  return train, test

In [ ]:

def build_rating_sparse_tensor(ratings_df):
  """
  Args:
    ratings_df: a pd.DataFrame with `user_id`, `ISBN` and `rating` columns.
  Returns:
    a tf.SparseTensor representing the ratings matrix.
  """
  indices = ratings_df[['User-ID', 'Book_id']].values
  values = ratings_df['Book-Rating'].values
  return tf.SparseTensor(
      indices=indices,
      values=values,
      dense_shape=[users.shape[0], all_books_final.shape[0]])

In [ ]:
def sparse_mean_square_error(sparse_ratings, user_embeddings, book_embeddings):
  """
  Args:
    sparse_ratings: A SparseTensor rating matrix, of dense_shape [N, M]
    user_embeddings: A dense Tensor U of shape [N, k] where k is the embedding
      dimension, such that U_i is the embedding of user i.
    book_embeddings: A dense Tensor V of shape [M, k] where k is the embedding
      dimension, such that V_j is the embedding of movie j.
  Returns:
    A scalar Tensor representing the MSE between the true ratings and the
      model's predictions.
  """
  predictions = tf.gather_nd(
      tf.matmul(user_embeddings, book_embeddings, transpose_b=True),
      sparse_ratings.indices)
  loss = tf.losses.mean_squared_error(sparse_ratings.values, predictions)
  return loss

The CFModel helper class takes an object of 3 arguments
* Embeddings which is a dictionary of the User_ID and Book ISBNs and the related Tensors
* A loss calculation, in this case MSE on the sparse tensor and embedding spaces of the training dataset
* A dictionary of metrics calculated for the train and test datasets converted to a list

In [ ]:
class CFModel(object):
  """Simple class that represents a collaborative filtering model"""
  def __init__(self, embedding_vars, loss, metrics=None):
    """Initializes a CFModel.
    Args:
      embedding_vars: A dictionary of tf.Variables.
      loss: A float Tensor. The loss to optimize.
      metrics: optional list of dictionaries of Tensors. The metrics in each
        dictionary will be plotted in a separate figure during training.
    """
    self._embedding_vars = embedding_vars
    self._loss = loss
    self._metrics = metrics
    self._embeddings = {k: None for k in embedding_vars}
    self._session = None

  @property
  def embeddings(self):
    """The embeddings dictionary."""
    return self._embeddings

  def train(self, num_iterations=100, learning_rate=1.0, plot_results=True,
            optimizer=tf.train.GradientDescentOptimizer):
    """Trains the model.
    Args:
      iterations: number of iterations to run.
      learning_rate: optimizer learning rate.
      plot_results: whether to plot the results at the end of training.
      optimizer: the optimizer to use. Default to GradientDescentOptimizer.
    Returns:
      The metrics dictionary evaluated at the last iteration.
    """
    with self._loss.graph.as_default():
      opt = optimizer(learning_rate)
      train_op = opt.minimize(self._loss)
      local_init_op = tf.group(
          tf.variables_initializer(opt.variables()),
          tf.local_variables_initializer())
      if self._session is None:
        self._session = tf.Session()
        with self._session.as_default():
          self._session.run(tf.global_variables_initializer())
          self._session.run(tf.tables_initializer())
          tf.train.start_queue_runners()

    with self._session.as_default():
      local_init_op.run()
      iterations = []
      metrics = self._metrics or ({},)
      metrics_vals = [collections.defaultdict(list) for _ in self._metrics]

      # Train and append results.
      for i in range(num_iterations + 1):
        _, results = self._session.run((train_op, metrics))
        if (i % 10 == 0) or i == num_iterations:
          print("\r iteration %d: " % i + ", ".join(
                ["%s=%f" % (k, v) for r in results for k, v in r.items()]),
                end='')
          iterations.append(i)
          for metric_val, result in zip(metrics_vals, results):
            for k, v in result.items():
              metric_val[k].append(v)

      for k, v in self._embedding_vars.items():
        self._embeddings[k] = v.eval()

      if plot_results:
        # Plot the metrics.
        num_subplots = len(metrics)+1
        fig = plt.figure()
        fig.set_size_inches(num_subplots*10, 8)
        for i, metric_vals in enumerate(metrics_vals):
          ax = fig.add_subplot(1, num_subplots, i+1)
          for k, v in metric_vals.items():
            ax.plot(iterations, v, label=k)
          ax.set_xlim([1, num_iterations])
          ax.legend()
      return results

The CFM function takes as input the ratings dataframe, the dimensions of the embedding vectors, and the standard deviation of the random initial embeddings <br/>
The code first creates 2 subsets of data for the test and train splits <br/>
It then creates the sparse tensor for each split <br/>
The embedding spaces U and V are created using the provided dimensions and a normal distribution from the train data split <br/>
Loss is calculated for MSE on the sparse tensor and embedding spaces and saved to a metrics dictionary
Finally the embeddings are labelled and saved to a dictionary. 
The build_model function returns the model created by the helper function

In [ ]:
def build_model(ratings, embedding_dim=3, init_stddev=1.):
  """
  Args:
    ratings: a DataFrame of the ratings
    embedding_dim: the dimension of the embedding vectors.
    init_stddev: float, the standard deviation of the random initial embeddings.
  Returns:
    model: a CFModel.
  """
  # Split the ratings DataFrame into train and test.
  train_ratings, test_ratings = split_dataframe(ratings)
  # SparseTensor representation of the train and test datasets.
  A_train = build_rating_sparse_tensor(train_ratings)
  A_test = build_rating_sparse_tensor(test_ratings)
  # Initialize the embeddings using a normal distribution.
  U = tf.Variable(tf.random_normal(
      [A_train.dense_shape[0], embedding_dim], stddev=init_stddev))
  V = tf.Variable(tf.random_normal(
      [A_train.dense_shape[1], embedding_dim], stddev=init_stddev))
  train_loss = sparse_mean_square_error(A_train, U, V)
  test_loss = sparse_mean_square_error(A_test, U, V)
  metrics = {
      'train_error': train_loss,
      'test_error': test_loss
  }
  embeddings = {
      "user_id": U,
      "ISBN": V
  }
  return CFModel(embeddings, train_loss, [metrics])

Building the Collaborative Filtering Matrix Model

In [ ]:
matrix_model=build_model(ratings, embedding_dim=3, init_stddev=0.5)
#matrix_model.train(num_iterations=100, learning_rate=0.1, plot_results=True)

## Collaborative Filtering-autoencoder Deep Neural Network

## Recommendation System

## Performance Evaluation

## Conclusion